# Plutus bridge from Charli3 oracle to Marlowe contracts


**Executive Summary**

We demonstrate the operation of a general-purpose bridge between Marlowe contracts and the Charli3 oracle. This "oracle bridge" is a small Plutus validator script that verifies that the price from the Charli3 oracle feed's reference UTXO is correctly input into the price `Choice` for the Marlowe contract. The oracle bridge enforces the following security guarantees:
1. The bridge will only read the price data from the oracle feed reference UTXO that contains the specified Charli3 `OracleFeed` token's policy ID.
2. The bridge ensures that the `ChoiceId` of the Marlowe contract `IChoice` action matches the specified name of the Charli3 oracle feed (in this example `Charli3 ADAUSD`).
3. The bridge checks that the `ChoiceNum` of the Marlowe contract `IChoice` action matches the prices datum in the oracle feed reference UTXO.
4. The bridge checks for the presence of the "thread" role token in the Marlowe contract instance.
5. The bridge holds the `Oracle` role token for the Marlowe contract instance, ensuring that Marlowe will only accept oracle input from that particular bridge instance.
6. The contract is uses the Marlowe validator.
7. The Cardano ledger and Charli3 semantics ensure that a "stale" or dated reference UTXO is not used by the bridge.

The diagram below shows the oracle bridge's interaction with the Charli3 reference UTXO and with the Marlowe contract. The Marlowe contract is not aware of the bridge's nature aside from the fact that the bridge holds the `Oracle` role token that authorizes the `IChoice` that inputs the Charli3 ADA/USD price into the Marlowe contract.

![Oracle bridge's interaction with Charli3 and Marlowe](bridge.png)

The demonstration uses the simplest possible Marlowe contract that interacts with an oracle. The contract simply waits for price input from the oracle bridge and then waits for a notification to close the contract. (Note that the price input and notification cannot take place in the same transaction because of Marlowe's guard against double-satisfaction attacks.) A more realistic Marlowe contract would use the price input in its payment logic.

![Simple Marlowe oracle](contract.png)

[A video demonstration demonstrating the oracle bridge on Cardano mainnet](https://youtu.be/_9DgXb323CE) is available.

References:
- [Source code for oracle bridge](https://github.com/input-output-hk/marlowe-plutus/tree/PLT-7535/marlowe-plutus/charli3)
- [Marlowe on Cardano](https://github.com/input-output-hk/marlowe-cardano)
- [Charli3 documentation](https://docs.charli3.io/)
- [Charli3 oracle-datum-lib](https://github.com/Charli3-Official/oracle-datum-lib)

## Utility functions

### Find the tip of the Cardano node

In [1]:
function node_tip {
cardano-cli query tip \
  "${MAGIC[@]}" \
| jq -r .slot
}

### Select a UTXO for use in a transaction

In [2]:
function select_utxo {
marlowe-cli util select \
  "${MAGIC[@]}" \
  --lovelace-only "$1" \
  "$PAYMENT_ADDR" \
| sed -n -e 's/^TxIn "\(.*\)" (TxIx \(.*\))$/\1#\2/;1p'
}

### Select a UTXO for the thread role token

In [3]:
function select_thread_utxo {
marlowe-cli util select \
  "${MAGIC[@]}" \
  --asset-only "$ROLES_CURRENCY.$THREAD_ROLE" \
  "$PAYMENT_ADDR" \
| sed -n -e 's/^TxIn "\(.*\)" (TxIx \(.*\))$/\1#\2/;1p'
}

### Select a UTXO for the oracle bridge role token

In [4]:
function select_bridge_utxo {
marlowe-cli util select \
  "${MAGIC[@]}" \
  --asset-only "$ROLES_CURRENCY.$BRIDGE_ROLE" \
  "$PAYMENT_ADDR" \
| sed -n -e 's/^TxIn "\(.*\)" (TxIx \(.*\))$/\1#\2/;1p'
}

### Sign and submit a transaction

In [5]:
function tx_submit {
cardano-cli transaction sign \
  "${MAGIC[@]}" \
  --signing-key-file "$PAYMENT_SKEY" \
  --tx-body-file "$1.unsigned" \
  --out-file "$1.signed"
cardano-cli transaction submit \
  "${MAGIC[@]}" \
  --tx-file "$1.signed" \
| grep -v 'Transaction successfully submitted.'
TMP_TXID=$(cardano-cli transaction txid --tx-file "$1.signed")
while [[ $(cardano-cli query utxo "${MAGIC[@]}" --tx-in "$TMP_TXID#0" --out-file /dev/stdout | jq length) == 0 ]]; do sleep 5s; done
echo "$TMP_TXID"
}

## Preliminaries

### Network

Select the network.

In [6]:
NETWORK=mainnet

Connect to the node.

In [7]:
if [[ "$NETWORK" == "mainnet" ]]
then

export CARDANO_NODE_SOCKET_PATH=mainnet/node.socket
unset CARDANO_TESTNET_MAGIC
MAGIC=(--mainnet)
CARDANOSCAN_URL=https://cardanoscan.io

else

export CARDANO_NODE_SOCKET_PATH=preprod/node.socket
export CARDANO_TESTNET_MAGIC=1
MAGIC=(--testnet-magic 1)
CARDANOSCAN_URL=https://preprod.cardanoscan.io

fi

Retrieve the protocol parameters.

In [8]:
cardano-cli query protocol-parameters \
  "${MAGIC[@]}" \
  --out-file network.protocol

### Oracle bridge

Use the Charli3 oracle bridge for Marlowe.

In [9]:
BRIDGE_EXE=marlowe-charli3

Serialise the Plutus script.

In [10]:
$BRIDGE_EXE

Mainnet: True
Validator hash: 5cf63b098791dad54c986b0b24f9c645de3226a7ccbd71fd64f22518
Validator file: charli3.plutus


Compute the address of the oracle bridge.

In [11]:
BRIDGE_ADDR=$(
cardano-cli address build \
  "${MAGIC[@]}" \
  --payment-script-file charli3.plutus \
)
echo "BRIDGE_ADDR = $BRIDGE_ADDR"

BRIDGE_ADDR = addr1w9w0vwcfs7ga442vnp4skf8ecezauv3x5lxt6u0avnez2xqfa52pa


### Payment credential

Set up a key for funding transactions.

In [12]:
if [[ "$NETWORK" == "mainnet" ]]
then

PAYMENT_ADDR=addr1q9drnfv8x6uyz7enyfckc4ktwt45p9eqv0yklqlnzvgv7synjlzdfawd9kvva673fht7737e3r5j322v7090uqhpn0wqqr3sw0
PAYMENT_SKEY=service-4.skey

else

PAYMENT_ADDR=$(cat payment.testnet.address)
PAYMENT_SKEY=payment.skey

fi

echo "PAYMENT_ADDR = $PAYMENT_ADDR"

PAYMENT_ADDR = addr1q9drnfv8x6uyz7enyfckc4ktwt45p9eqv0yklqlnzvgv7synjlzdfawd9kvva673fht7737e3r5j322v7090uqhpn0wqqr3sw0


### Marlowe validator script

Check the hash of the Marlowe semantics validator script.

In [13]:
marlowe-cli contract validator \
  "${MAGIC[@]}" \
  --out-file /dev/null \
  --print-hash

addr1wyhdyccahvnheppng3fut3phhp3jt5m37zp4529ezz535ms353w0f

Validator hash: "2ed2631dbb277c84334453c5c437b86325d371f0835a28b910a91a6e"


Compute the address of the Marlowe interpreter.

In [14]:
MARLOWE_ADDR=$(marlowe-cli contract address "${MAGIC[@]}")
echo "MARLOWE_ADDR = $MARLOWE_ADDR"

MARLOWE_ADDR = addr1wyhdyccahvnheppng3fut3phhp3jt5m37zp4529ezz535ms353w0f


Find the reference UTXO for the Marlowe interpreter.

In [15]:
if [[ "$NETWORK" == "mainnet" ]]
then

UTXO_MARLOWE_SCRIPT=$(
marlowe-cli transaction find-published \
   "${MAGIC[@]}" \
   --at-address addr1z9l4w7djneh0kss4drg2php6ynflsvmal7x3w5nrc95uvhz7e4q926apsvcd6kn33cpx95k8jsmrj7v0k62rczvz8urqrl2z0l \
  2> /dev/null \
| jq -r .marlowe.txIn
)

else

UTXO_MARLOWE_SCRIPT=$(
marlowe-cli transaction find-published \
   "${MAGIC[@]}" \
  2> /dev/null \
| jq -r .marlowe.txIn
)

fi

echo "UTXO_MARLOWE_SCRIPT = $UTXO_MARLOWE_SCRIPT"

UTXO_MARLOWE_SCRIPT = 672399f7d551d6e06fda70769f830e4e3783495c6250567c6ae97ecc788ad5a4#1


### Datums and redeemers

Create redeemers for controlling the oracle bridge.

In [16]:
echo '{"constructor": 0, "fields": []}' > terminate.redeemer
echo '{"constructor": 1, "fields": []}' > continue.redeemer

Create the datum for the oracle bridge.

In [17]:
THREAD_ROLE=Thread
echo '{"bytes": "'$(echo -n $THREAD_ROLE | basenc --base16)'"}' > thread.datum
THREAD_HEX=$(jq -r .bytes thread.datum | tr '[:upper:]' '[:lower:]')
echo "THREAD_HEX = $THREAD_HEX"

THREAD_HEX = 546872656164


### Constants

One ada is one million lovelace.

In [18]:
ADA=1000000

### Minting

Compute the policy ID for the Marlowe roles currency.

In [19]:
ROLES_CURRENCY=$(
marlowe-cli util mint \
   "${MAGIC[@]}" \
  --issuer "$PAYMENT_ADDR:$PAYMENT_SKEY" \
  --out-file /dev/null \
  "$THREAD_ROLE:$PAYMENT_ADDR" \
  2> /dev/null
)
echo "ROLES_CURRENCY = $ROLES_CURRENCY"

ROLES_CURRENCY = bd0dddf6d44a37ba142096f71d281159544378c4f17887a39797c002


#### Mint the thread token, if necessary

Mint the thread token to uniquely identify the Marlowe contract.

In [20]:
if [[ $(select_thread_utxo) == "" ]]
then
echo -n "Minted with policy ID"
marlowe-cli util mint \
   "${MAGIC[@]}" \
  --issuer "$PAYMENT_ADDR:$PAYMENT_SKEY" \
  --out-file /dev/null \
  --submit 600s \
  "$THREAD_ROLE:$PAYMENT_ADDR" \
  2> /dev/null
else
  echo "Already minted."
fi

Minted with policy IDbd0dddf6d44a37ba142096f71d281159544378c4f17887a39797c002


#### Mint the oracle bridge role token, if necessary

Mint the role token for the oracle bridge.

In [21]:
BRIDGE_ROLE=Oracle
BRIDGE_HEX=$(echo -n "$BRIDGE_ROLE" | basenc --base16 | tr '[:upper:]' '[:lower:]')
echo "BRIDGE_HEX = $BRIDGE_HEX"

BRIDGE_HEX = 4f7261636c65


In [22]:
if [[ $(select_bridge_utxo) == "" ]]
then
echo -n "Minted with policy ID"
marlowe-cli util mint \
   "${MAGIC[@]}" \
  --issuer "$PAYMENT_ADDR:$PAYMENT_SKEY" \
  --out-file /dev/null \
  --submit 600s \
  "$BRIDGE_ROLE:$PAYMENT_ADDR" \
  2> /dev/null
else
  echo "Already minted."
fi

Minted with policy IDbd0dddf6d44a37ba142096f71d281159544378c4f17887a39797c002


### Publish the oracle bridge

Publish the reference script for the oracle bridge.

In [23]:
if [[ "$NETWORK" == "mainnet" ]]
then
  PUBLISH_ADDR=addr1q9mtvww5dl3gy06xawnk6g77vn0ealu7vrex3t7dd50gqn5njlzdfawd9kvva673fht7737e3r5j322v7090uqhpn0wqjv7p02
else
  PUBLISH_ADDR=addr_test1qpmtvww5dl3gy06xawnk6g77vn0ealu7vrex3t7dd50gqn5njlzdfawd9kvva673fht7737e3r5j322v7090uqhpn0wq36rpr4
fi
PUBLISH_SKEY=service-5.skey
echo "PUBLISH_ADDR = $PUBLISH_ADDR"

PUBLISH_ADDR = addr1q9mtvww5dl3gy06xawnk6g77vn0ealu7vrex3t7dd50gqn5njlzdfawd9kvva673fht7737e3r5j322v7090uqhpn0wqjv7p02


In [24]:
cardano-cli transaction build \
  --babbage-era \
  "${MAGIC[@]}" \
  --protocol-params-file network.protocol \
  --tx-in "$(select_utxo $((50 * ADA)))" \
  --tx-out "$PUBLISH_ADDR+$((30 * ADA))" \
    --tx-out-reference-script-file charli3.plutus \
  --change-address "$PAYMENT_ADDR" \
  --out-file publish.unsigned

Estimated transaction fee: Lovelace 444637


In [25]:
UTXO_BRIDGE_SCRIPT="$(tx_submit publish)#1"
echo "UTXO_BRIDGE_SCRIPT = $UTXO_BRIDGE_SCRIPT"

UTXO_BRIDGE_SCRIPT = 40d675058587801741eac693bf7007400230712834819d3b583a0ade9183b14d#1


## Test using the oracle bridge with the Marlowe contract

### Create the oracle bridge

Find the UTXO with the role token for the oracle bridge.

In [26]:
UTXO_BRIDGE=$(select_bridge_utxo)
echo "UTXO_BRIDGE = $UTXO_BRIDGE"

UTXO_BRIDGE = 2d6fd9b6a97cc294faa7f0b166cb2e9a531577902099bd4e8239e94e71f854fa#2


Build and submit the transaction to create the oracle bridge.

In [27]:
cardano-cli transaction build \
  --babbage-era \
  "${MAGIC[@]}" \
  --protocol-params-file network.protocol \
  --tx-in "$(select_utxo $((10 * ADA)))" \
  --tx-in "$UTXO_BRIDGE" \
  --tx-out "$BRIDGE_ADDR+$((2 * ADA))+1 $ROLES_CURRENCY.$BRIDGE_HEX" \
    --tx-out-datum-embed-file thread.datum \
  --change-address "$PAYMENT_ADDR" \
  --out-file test-0.unsigned

Estimated transaction fee: Lovelace 178349


In [28]:
TX_0=$(tx_submit test-0)
echo "$CARDANOSCAN_URL/transaction/$TX_0?tab=utxo"

https://cardanoscan.io/transaction/171669a92a19b5db819313f0c7eb8939a6476fc3eec541ddf8d70392245aae02?tab=utxo


In [29]:
UTXO_BRIDGE="$TX_0#1"
echo "UTXO_BRIDGE = $UTXO_BRIDGE"

UTXO_BRIDGE = 171669a92a19b5db819313f0c7eb8939a6476fc3eec541ddf8d70392245aae02#1


### Design the contract

Create the Marlowe contract for testing the oracle.

In [30]:
yaml2json << EOI > test-1.contract
when:
- case:
    choose_between:
    - from: 1
      to: 1000000000
    for_choice:
      choice_name: Charli3 ADAUSD  # Currently, this is hard-coded in the oracle bridge's validator script.
      choice_owner:
        role_token: $BRIDGE_ROLE
  then:
    when:
    - case:
        notify_if: true
      then: close
    timeout: $((1000 * $(date -d "$(date -u) + 90 minutes" +%s)))  # Ninety minutes from now.
    timeout_continuation: close
timeout: $((1000 * $(date -d "$(date -u) + 89 minutes" +%s)))  # Ninety minutes from now.
timeout_continuation: close
EOI
cat test-1.contract

{"timeout":1694905048000,"timeout_continuation":"close","when":[{"case":{"choose_between":[{"from":1,"to":1000000000}],"for_choice":{"choice_name":"Charli3 ADAUSD","choice_owner":{"role_token":"Oracle"}}},"then":{"timeout":1694905108000,"timeout_continuation":"close","when":[{"case":{"notify_if":true},"then":"close"}]}}]}


#### Set the initial state

The initial state must contain the thread token and the required minimum ada.

In [31]:
INITIAL_LOVELACE=$((2 * ADA))
yaml2json << EOI > test-1.state
accounts:
- - - address: $PAYMENT_ADDR
    - currency_symbol: ''
      token_name: ''
  - $INITIAL_LOVELACE
- - - address: $PAYMENT_ADDR
    - currency_symbol: $ROLES_CURRENCY
      token_name: $THREAD_ROLE
  - 1
choices: []
boundValues: []
minTime: 1
EOI
cat test-1.state

{"accounts":[[[{"address":"addr1q9drnfv8x6uyz7enyfckc4ktwt45p9eqv0yklqlnzvgv7synjlzdfawd9kvva673fht7737e3r5j322v7090uqhpn0wqqr3sw0"},{"currency_symbol":"","token_name":""}],2000000],[[{"address":"addr1q9drnfv8x6uyz7enyfckc4ktwt45p9eqv0yklqlnzvgv7synjlzdfawd9kvva673fht7737e3r5j322v7090uqhpn0wqqr3sw0"},{"currency_symbol":"bd0dddf6d44a37ba142096f71d281159544378c4f17887a39797c002","token_name":"Thread"}],1]],"boundValues":[],"choices":[],"minTime":1}


### Build and submit the Marlowe creation transaction

Sadly, we cannot yet use Marlowe Runtime for executing Marlowe contracts that utilize auxillary Plutus scripts like the oracle bridge. We'll have to manually construct the datums and redeemers using the high- and low-level commands in `marlowe-cli` and to manually build and submit the transactions with `cardano-cli`.

In [32]:
marlowe-cli run initialize \
  "${MAGIC[@]}" \
  --roles-currency "$ROLES_CURRENCY" \
  --contract-file test-1.contract \
  --state-file test-1.state \
  --out-file test-1.marlowe  

In [33]:
marlowe-cli contract datum \
  --roles-currency "$ROLES_CURRENCY" \
  --contract-file test-1.contract \
  --state-file test-1.state \
  --out-file test-1.datum

63a4a058514c8485bbf5b5e14b0033d4b5d06584c70e097d52e585ffdb927b76


Find the UTXO holding the thread token.

In [34]:
UTXO_THREAD=$(select_thread_utxo)
echo "UTXO_THREAD = $UTXO_THREAD"

UTXO_THREAD = 2d6fd9b6a97cc294faa7f0b166cb2e9a531577902099bd4e8239e94e71f854fa#1


Build and submit the transaction to create the Marlowe contract.

In [35]:
cardano-cli transaction build \
  --babbage-era \
  "${MAGIC[@]}" \
  --protocol-params-file network.protocol \
  --tx-in "$(select_utxo $((10 * ADA)))" \
  --tx-in "$UTXO_THREAD" \
  --tx-out "$MARLOWE_ADDR+$INITIAL_LOVELACE+1 $ROLES_CURRENCY.$THREAD_HEX" \
    --tx-out-datum-embed-file test-1.datum \
  --change-address "$PAYMENT_ADDR" \
  --out-file test-1.unsigned

Estimated transaction fee: Lovelace 194849


In [36]:
TX_1=$(tx_submit test-1)
echo "$CARDANOSCAN_URL/transaction/$TX_1?tab=utxo"
echo "https://$NETWORK.marlowescan.com/contractView?tab=info&contractId=$TX_1%231"

https://cardanoscan.io/transaction/9a04c856b7591d353a0a258ebeec335c544ba5ece1fc180b9124ca0d59b56c71?tab=utxo
https://mainnet.marlowescan.com/contractView?tab=info&contractId=9a04c856b7591d353a0a258ebeec335c544ba5ece1fc180b9124ca0d59b56c71%231


In [37]:
UTXO_MARLOWE="$TX_1#1"
echo "UTXO_MARLOWE = $UTXO_MARLOWE"

UTXO_MARLOWE = 9a04c856b7591d353a0a258ebeec335c544ba5ece1fc180b9124ca0d59b56c71#1


### Read the Charli3 oracle feed

Now find the reference input for the Charli3 ADAUSD oracle. We query Marlowe Runtime's `chain` schema to determine this.

In [38]:
export PGPASSWORD=$(cat psql.password)

if [[ "$NETWORK" == "mainnet" ]]
then
  CHARLI3_POLICY=30d7c4da385a1f5044261d27b6a22d46b645ca3567636df5edeb303d
else
  CHARLI3_POLICY=e4c846f0f87a7b4524d8e7810ed957c6b7f6e4e2e2e42d75ffe7b373
fi

psql -t -h 192.168.0.12 "$NETWORK" << EOI > charli3-adausd.json
select row_to_json(result)
  from (
    select
        encode(txid, 'hex') as txid
      , txix
      , encode(datumhash, 'hex') as datumhash
      , encode(datumbytes, 'hex') as datumbytes
      from chain.txout
        where (txid, txix) in (
          select txoutid, txoutix
            from chain.assetout
            inner join chain.asset
              on assetout.assetid = asset.id
            where policyid = '\x$CHARLI3_POLICY'
              and name = '\x4f7261636c6546656564'
          order by slotno desc
          limit 1
        )
  ) result
;
EOI

json2yaml charli3-adausd.json

datumbytes: d8799fd87b9fa3001a0003c4e7011b0000018a9fe38621021b0000018aa01a74a1ffff
datumhash: 7ec270f971d03f084c1c198a4f99e53dc8519430dcf4123d65b9e1c48fedd82e
txid: ba88b539c186e1723ade42be681f00f1bb3150f9fdf87cc93047d4c7de15983c
txix: 1


Format the UTXO for the Charli3 reference input.

In [39]:
UTXO_CHARLI3=$(jq -r '.txid + "#" + (.txix|tostring)' charli3-adausd.json)
echo "UTXO_CHARLI3 = $UTXO_CHARLI3"

UTXO_CHARLI3 = ba88b539c186e1723ade42be681f00f1bb3150f9fdf87cc93047d4c7de15983c#1


View the reference UTXO.

In [40]:
cardano-cli query utxo "${MAGIC[@]}" --tx-in "$UTXO_CHARLI3"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
ba88b539c186e1723ade42be681f00f1bb3150f9fdf87cc93047d4c7de15983c     1        2000000 lovelace + 1 30d7c4da385a1f5044261d27b6a22d46b645ca3567636df5edeb303d.4f7261636c6546656564 + TxOutDatumInline ReferenceTxInsScriptsInlineDatumsInBabbageEra (ScriptDataConstructor 0 [ScriptDataConstructor 2 [ScriptDataMap [(ScriptDataNumber 0,ScriptDataNumber 247015),(ScriptDataNumber 1,ScriptDataNumber 1694899602977),(ScriptDataNumber 2,ScriptDataNumber 1694903202977)]]])


Extract the inline datum in the reference UTXO.

In [41]:
cardano-cli query utxo "${MAGIC[@]}" --tx-in "$UTXO_CHARLI3" --out-file charli3.utxo

Extract the price from the reference UTXO's datum.

In [42]:
PRICE=$(jq -r '.['"$CHARLI3_UTXO"'].inlineDatum.fields[0].fields[0].map[0].v.int' charli3.utxo)
echo "PRICE = $(echo "scale=6; $PRICE / 1000000" | bc | sed 's/^\./0&/') USD/ADA" 

PRICE = 0.247015 USD/ADA


#### Build and submit the choice transaction

Set the choice of price in the input action to the Marlowe contract.

In [43]:
marlowe-cli run prepare \
  --marlowe-file test-1.marlowe \
  --out-file test-2.marlowe \
  --choice-name "Charli3 ADAUSD" \
  --choice-party "$BRIDGE_ROLE" \
  --choice-number "$PRICE" \
  --invalid-before "$((1000 * $(date -d "$(date -u) - 1 minutes" +%s)))" \
  --invalid-hereafter "$((1000 * $(date -d "$(date -u) + 5 minutes" +%s)))" \
  2> /dev/null

TransactionInput {txInterval = (POSIXTime {getPOSIXTime = 1694899893000},POSIXTime {getPOSIXTime = 1694900253999}), txInputs = [NormalInput (IChoice (ChoiceId "Charli3 ADAUSD" "Oracle") 247015)]}


Compute the datum and redeemer for the Marlowe transaction.

In [44]:
jq '.tx.inputs[0]' test-2.marlowe > test-2.input
marlowe-cli contract redeemer \
  --input-file test-2.input \
  --out-file test-2.redeemer

In [45]:
jq .tx.contract test-2.marlowe > test-2.contract
jq .tx.state test-2.marlowe > test-2.state
marlowe-cli contract datum \
  --roles-currency "$ROLES_CURRENCY" \
  --contract-file test-2.contract \
  --state-file test-2.state \
  --out-file test-2.datum

e97068048f721e41d3879be828ef2bbea2f7619f50c56db02f97490a6a2f30c4


Compute the transaction cost.

In [46]:
cardano-cli transaction build \
  --babbage-era \
  "${MAGIC[@]}" \
  --protocol-params-file network.protocol \
  --tx-in "$(select_utxo $((10 * ADA)))" \
  --tx-in "$UTXO_BRIDGE" \
    --spending-tx-in-reference "$UTXO_BRIDGE_SCRIPT" \
    --spending-plutus-script-v2 \
    --spending-reference-tx-in-datum-file thread.datum \
    --spending-reference-tx-in-redeemer-file continue.redeemer \
  --tx-in "$UTXO_MARLOWE" \
    --spending-tx-in-reference "$UTXO_MARLOWE_SCRIPT" \
    --spending-plutus-script-v2 \
    --spending-reference-tx-in-datum-file test-1.datum \
    --spending-reference-tx-in-redeemer-file test-2.redeemer \
  --read-only-tx-in-reference "$UTXO_CHARLI3" \
  --tx-in-collateral "$(select_utxo $((10 * ADA)))" \
  --tx-out "$BRIDGE_ADDR+$((2 * ADA))+1 $ROLES_CURRENCY.$BRIDGE_HEX" \
    --tx-out-datum-embed-file thread.datum \
  --tx-out "$MARLOWE_ADDR+$INITIAL_LOVELACE+1 $ROLES_CURRENCY.$THREAD_HEX" \
    --tx-out-datum-embed-file test-2.datum \
  --change-address "$PAYMENT_ADDR" \
  --invalid-before "$(jq -r '.tx.range[0]' test-2.marlowe)" \
  --invalid-hereafter "$(jq -r '.tx.range[1]' test-2.marlowe)" \
  --calculate-plutus-script-cost test-2.cost
json2yaml test-2.cost

Estimated transaction fee: Lovelace 973557
- executionUnits:
    memory: 6958112
    steps: 1812041878
  lovelaceCost: 532132
  scriptHash: 2ed2631dbb277c84334453c5c437b86325d371f0835a28b910a91a6e
- executionUnits:
    memory: 2735374
    steps: 770111224
  lovelaceCost: 213357
  scriptHash: 5cf63b098791dad54c986b0b24f9c645de3226a7ccbd71fd64f22518


Compare this to the execution cost limits for the protocol.

In [47]:
jq .maxTxExecutionUnits network.protocol | json2yaml

memory: 14000000
steps: 10000000000


Compute the fraction of Plutus memory that were used.

In [48]:
FRACTION_MEMORY=$(
echo "scale=4; ($(jq -r '.[0].executionUnits.memory' test-2.cost) + $(jq -r '.[1].executionUnits.memory' test-2.cost)) / $(jq .maxTxExecutionUnits.memory network.protocol)" \
| bc \
| sed -e 's/^\./0&/' \
)
echo "FRACTION_MEMORY = $FRACTION_MEMORY"

FRACTION_MEMORY = 0.6923


Compute the fraction of Plutus steps that were used.

In [49]:
FRACTION_STEPS=$(
echo "scale=4; ($(jq -r '.[0].executionUnits.steps' test-2.cost) + $(jq -r '.[1].executionUnits.steps' test-2.cost)) / $(jq .maxTxExecutionUnits.steps network.protocol)" \
| bc \
| sed -e 's/^\./0&/' \
)
echo "FRACTION_STEPS = $FRACTION_STEPS"

FRACTION_STEPS = 0.2582


Build and submit the transaction to read the price from the Charli3 oracle into the Marlowe contract, using the oracle bridge.

In [50]:
cardano-cli transaction build \
  --babbage-era \
  "${MAGIC[@]}" \
  --protocol-params-file network.protocol \
  --tx-in "$(select_utxo $((10 * ADA)))" \
  --tx-in "$UTXO_BRIDGE" \
    --spending-tx-in-reference "$UTXO_BRIDGE_SCRIPT" \
    --spending-plutus-script-v2 \
    --spending-reference-tx-in-datum-file thread.datum \
    --spending-reference-tx-in-redeemer-file continue.redeemer \
  --tx-in "$UTXO_MARLOWE" \
    --spending-tx-in-reference "$UTXO_MARLOWE_SCRIPT" \
    --spending-plutus-script-v2 \
    --spending-reference-tx-in-datum-file test-1.datum \
    --spending-reference-tx-in-redeemer-file test-2.redeemer \
  --read-only-tx-in-reference "$UTXO_CHARLI3" \
  --tx-in-collateral "$(select_utxo $((10 * ADA)))" \
  --tx-out "$BRIDGE_ADDR+$((2 * ADA))+1 $ROLES_CURRENCY.$BRIDGE_HEX" \
    --tx-out-datum-embed-file thread.datum \
  --tx-out "$MARLOWE_ADDR+$INITIAL_LOVELACE+1 $ROLES_CURRENCY.$THREAD_HEX" \
    --tx-out-datum-embed-file test-2.datum \
  --change-address "$PAYMENT_ADDR" \
  --invalid-before "$(jq -r '.tx.range[0]' test-2.marlowe)" \
  --invalid-hereafter "$(jq -r '.tx.range[1]' test-2.marlowe)" \
  --out-file test-2.unsigned

Estimated transaction fee: Lovelace 973557


In [51]:
TX_2=$(tx_submit test-2)
echo "$CARDANOSCAN_URL/transaction/$TX_2?tab=utxo"
echo "https://$NETWORK.marlowescan.com/contractView?tab=tx&contractId=$TX_1%231&transactionId=$TX_2"

https://cardanoscan.io/transaction/7c51f613cdf181fda2967847eb39b2d42a41eafa9f4e45c6869362590c575548?tab=utxo
https://mainnet.marlowescan.com/contractView?tab=tx&contractId=9a04c856b7591d353a0a258ebeec335c544ba5ece1fc180b9124ca0d59b56c71%231&transactionId=7c51f613cdf181fda2967847eb39b2d42a41eafa9f4e45c6869362590c575548


In [52]:
UTXO_BRIDGE="$TX_2#1"
UTXO_MARLOWE="$TX_2#2"
echo "UTXO_BRIDGE = $UTXO_BRIDGE"
echo "UTXO_MARLOWE = $UTXO_MARLOWE"

UTXO_BRIDGE = 7c51f613cdf181fda2967847eb39b2d42a41eafa9f4e45c6869362590c575548#1
UTXO_MARLOWE = 7c51f613cdf181fda2967847eb39b2d42a41eafa9f4e45c6869362590c575548#2


#### Build and submit the notify transaction

Advance the Marlowe contract through its final `Notify` step.

In [53]:
marlowe-cli run prepare \
  --marlowe-file test-2.marlowe \
  --out-file test-3.marlowe \
  --notify \
  --invalid-before "$((1000 * $(date -d "$(date -u) - 1 minutes" +%s)))" \
  --invalid-hereafter "$((1000 * $(date -d "$(date -u) + 5 minutes" +%s)))" \
  2> /dev/null  

TransactionInput {txInterval = (POSIXTime {getPOSIXTime = 1694900119000},POSIXTime {getPOSIXTime = 1694900479999}), txInputs = [NormalInput INotify]}


Compute the redeemer.

In [54]:
jq '.tx.inputs[0]' test-3.marlowe > test-3.input
marlowe-cli contract redeemer \
  --input-file test-3.input \
  --out-file test-3.redeemer

Build and submit the transaction.

In [55]:
cardano-cli transaction build \
  --babbage-era \
  "${MAGIC[@]}" \
  --protocol-params-file network.protocol \
  --tx-in "$(select_utxo $((10 * ADA)))" \
  --tx-in "$UTXO_MARLOWE" \
    --spending-tx-in-reference "$UTXO_MARLOWE_SCRIPT" \
    --spending-plutus-script-v2 \
    --spending-reference-tx-in-datum-file test-2.datum \
    --spending-reference-tx-in-redeemer-file test-3.redeemer \
  --tx-in-collateral "$(select_utxo $((10 * ADA)))" \
  --tx-out "$PAYMENT_ADDR+$INITIAL_LOVELACE+1 $ROLES_CURRENCY.$THREAD_HEX" \
  --change-address "$PAYMENT_ADDR" \
  --invalid-before "$(jq -r '.tx.range[0]' test-3.marlowe)" \
  --invalid-hereafter "$(jq -r '.tx.range[1]' test-3.marlowe)" \
  --out-file test-3.unsigned

Estimated transaction fee: Lovelace 526426


In [56]:
TX_3=$(tx_submit test-3)
echo "$CARDANOSCAN_URL/transaction/$TX_3?tab=utxo"
echo "https://$NETWORK.marlowescan.com/contractView?tab=tx&contractId=$TX_1%231&transactionId=$TX_3"

https://cardanoscan.io/transaction/5a9de9652eac84296a0f15d857068e8f54d0884e771618ccc68fc8e0cf92988b?tab=utxo
https://mainnet.marlowescan.com/contractView?tab=tx&contractId=9a04c856b7591d353a0a258ebeec335c544ba5ece1fc180b9124ca0d59b56c71%231&transactionId=5a9de9652eac84296a0f15d857068e8f54d0884e771618ccc68fc8e0cf92988b


### Shut down the oracle bridge

Build and submit the transaction to shut down the oracle bridge.

In [57]:
cardano-cli transaction build \
  --babbage-era \
  "${MAGIC[@]}" \
  --protocol-params-file network.protocol \
  --tx-in "$(select_utxo $((10 * ADA)))" \
  --tx-in "$UTXO_BRIDGE" \
    --spending-tx-in-reference "$UTXO_BRIDGE_SCRIPT" \
    --spending-plutus-script-v2 \
    --spending-reference-tx-in-datum-file thread.datum \
    --spending-reference-tx-in-redeemer-file terminate.redeemer \
  --tx-in-collateral "$(select_utxo $((10 * ADA)))" \
  --tx-out "$PAYMENT_ADDR+$((2 * ADA))+1 $ROLES_CURRENCY.$BRIDGE_HEX" \
  --change-address "$PAYMENT_ADDR" \
  --out-file test-4.unsigned

Estimated transaction fee: Lovelace 250777


In [58]:
TX_4=$(tx_submit test-4)
echo "$CARDANOSCAN_URL/transaction/$TX_4?tab=utxo"

https://cardanoscan.io/transaction/b39d39e98a536a6da7f36481adf68df6b3f4f33fd805b1b09cb9a4ca455547b1?tab=utxo


## Clean up

Consume the reference script UTXO for the oracle bridge.

In [59]:
marlowe-cli transaction simple \
  "${MAGIC[@]}" \
  --tx-in "$UTXO_BRIDGE_SCRIPT" \
  --change-address "$PAYMENT_ADDR" \
  --required-signer "$PUBLISH_SKEY" \
  --out-file /dev/null \
  --submit 600

TxId "13f2bfeca326e77c6b3bee98898b655746a0b15287e4179f439dc42533d07927"


Burn the role tokens.

In [60]:
marlowe-cli util burn \
  "${MAGIC[@]}" \
  --issuer "$PAYMENT_ADDR:$PAYMENT_SKEY" \
  --token-provider "$PAYMENT_ADDR:$PAYMENT_SKEY" \
  --out-file /dev/null \
  --submit 600s \
  2> /dev/null

bd0dddf6d44a37ba142096f71d281159544378c4f17887a39797c002
